# Train Code for Convolutional Neural Networks  
CNNs are trained using this notebook

## The Data

In [1]:
import utils
reload(utils)

Using TensorFlow backend.


<module 'utils' from 'utils.pyc'>

In [2]:
(X_train, Y_train), (X_test, Y_test) = utils.load_data()

In [3]:
print X_train.shape
print X_test.shape
print Y_train.shape
print Y_test.shape

(50000, 3, 32, 32)
(10000, 3, 32, 32)
(50000, 10)
(10000, 10)


## The Model  
We use a 'typical' CNN with Max-Pooling layers to predict classes in CIFAR-10 dataset.  

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

Optionally, install [Hualos](https://github.com/fchollet/hualos) and use it to visualize the loss function live.  

In [5]:
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')

### Define the model

In [64]:
pool_size = (2, 2)
strides = (2, 2)

model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(3,32,32)))  # Layer 1
model.add(Convolution2D(32, 3, 3, activation='relu'))  # Layer 2
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))  # Layer 3
model.add(Dropout(0.25))  # Layer 4

model.add(Convolution2D(64, 3, 3, activation='relu'))  # Layer 5
model.add(Convolution2D(64, 3, 3, activation='relu'))  # Layer 6
model.add(MaxPooling2D(pool_size=pool_size, strides=strides))  # Layer 7
model.add(Dropout(0.25))  # Layer 8

model.add(Flatten())  # Layer 9

model.add(Dense(512, activation='relu'))  # Layer 10
model.add(Dropout(0.5))  # Layer 11
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

Or load saved model

In [6]:
model = utils.load_model('model-1')

### Train  the model

In [65]:
data_augmentation = False
nb_epoch = 50
rounds = 19
batch_size = 128
samples_per_epoch = X_train.shape[0] * rounds

In [66]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, 
              show_accuracy=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    # Only need fit() if we are using featurewise_center, featurewise_std_normalizaiton, or zca_whitening
    # datagen.fit(X_train, rounds=rounds)

    # fit the model one batch at a time using the original and augmented sets
    for e in range(nb_epoch):
        print 'Epoch', e
        model.fit(X_train, Y_train, nb_epoch=1, batch_size=batch_size, verbose=1, 
                  show_accuracy=True)
        model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                            samples_per_epoch=samples_per_epoch,
                            nb_epoch=1, show_accuracy=True,
                            nb_worker=1)

Not using data augmentation.
Epoch 1/50
50000/50000 [==============================] - 29s - loss: 1.9276 - acc: 0.2883    
Epoch 2/50
50000/50000 [==============================] - 28s - loss: 1.5404 - acc: 0.4348    
Epoch 3/50
50000/50000 [==============================] - 28s - loss: 1.4017 - acc: 0.4914    
Epoch 4/50
50000/50000 [==============================] - 29s - loss: 1.3072 - acc: 0.5295    
Epoch 5/50
50000/50000 [==============================] - 29s - loss: 1.2235 - acc: 0.5626    
Epoch 6/50
50000/50000 [==============================] - 28s - loss: 1.1600 - acc: 0.5864    
Epoch 7/50
50000/50000 [==============================] - 28s - loss: 1.1093 - acc: 0.6074    
Epoch 8/50
50000/50000 [==============================] - 29s - loss: 1.0567 - acc: 0.6239    
Epoch 9/50
50000/50000 [==============================] - 29s - loss: 1.0114 - acc: 0.6424    
Epoch 10/50
50000/50000 [==============================] - 29s - loss: 0.9744 - acc: 0.6568    
Epoch 11/50
50000/50

In [ ]:
model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, 
          show_accuracy=True)

### Test the model

In [7]:
loss, accuracy = model.evaluate(X_test, Y_test, show_accuracy=True)
print "Loss:", loss
print "Accuracy:", accuracy

10000/10000 [==============================] - 1s     
Loss: 0.589300656366
Accuracy: 0.8103


### Save the model

In [68]:
utils.save_model(model, 'model-2')

True